In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

import torch
from torch import nn
import torch.nn.functional as F

drive.mount('/content/gdrive', force_remount=True)
filepath = '/content/gdrive/MyDrive' + '/Colab Notebooks/csv/'
mnist = np.load(filepath + 'mnist.npz')

x_train = (mnist['x_train'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_train = mnist['y_train']
x_test = (mnist['x_test'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_test = mnist['y_test']
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/gdrive
(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 256, bias=True)
        self.fc2 = nn.Linear(256, 256, bias=True)
        self.fc3 = nn.Linear(256, 10, bias=True)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
from torch.optim import SGD

model = MLP()
criterion = nn.CrossEntropyLoss()

opti = SGD(model.parameters(), lr=1e-3)

In [6]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(torch.FloatTensor(x_train), torch.LongTensor(y_train))
batch_size = 64
shuffle = True
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

In [8]:
epoch = 10

losses = []
for i in  range(epoch):
    correct = 0
    for data, target in train_dataloader:
        data = data.view(-1, 28*28)

        output = model(data)
        loss = criterion(output, target)

        opti.zero_grad()
        loss.backward()
        opti.step()

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
    print(100.0 * correct / len(train_dataloader.dataset))

45.26
73.815
81.61333333333333
85.05333333333333
87.07333333333334
88.23833333333333
89.005
89.49
89.92666666666666
90.295


In [11]:
test_dataset = TensorDataset(torch.FloatTensor(x_test), torch.LongTensor(y_test))
batch_size =64
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [12]:
correct = 0
for data, target in test_dataloader:
    data = data.view(-1, 28*28)
    output = model(data)

    pred = output.max(1, keepdim=True)[1]
    correct += pred.eq(target.view_as(pred)).sum().item()
print(100.0 * correct / len(test_dataloader.dataset))

90.46833333333333
